In [1]:
import numpy as np
import pandas as pd
import os
import sys
import keras
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline


PATH = "/kaggle/input/tsp-cv"
PATH_TRAIN = os.path.join(PATH, "train.csv")
PATH_TEST = os.path.join(PATH , 'test.csv')
df_train = pd.read_csv(PATH_TRAIN)
df_test = pd.read_csv(PATH_TEST)

In [2]:
TRAIN_PCT = 0.78
TRAIN_CUT = int(len(df_train) * TRAIN_PCT)

df_train_cut = df_train[0:TRAIN_CUT]
df_validate_cut = df_train[TRAIN_CUT:]

In [3]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

WIDTH = 299
HEIGHT = 299

training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  #vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=32, # Keeping the training batch size small USUALLY increases performance
        class_mode='raw')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=256, # Make the validation batch size as large as you have memory for
        class_mode='raw')


test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=PATH,
        x_col="filename",
    
        batch_size=1,
        shuffle=False,
        target_size=(HEIGHT, WIDTH),
        class_mode=None)

Found 12494 validated image filenames.
Found 3524 validated image filenames.
Found 4005 validated image filenames.


In [4]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow import keras
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(HEIGHT, WIDTH, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='relu'))
model.compile(loss = 'mean_squared_error', 
              optimizer=keras.optimizers.Adam(1e-5), 
              metrics=[RootMeanSquaredError(name="rmse")])

STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto',
        restore_best_weights=True)

history = model.fit(train_generator, epochs=120, steps_per_epoch=50, 
                    validation_data = val_generator, callbacks=[monitor],
                    verbose = 1, validation_steps=STEP_SIZE_VALID)

2022-04-16 21:51:16.203392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 21:51:16.298974: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 21:51:16.299794: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 21:51:16.301304: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/120


2022-04-16 21:51:20.880830: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


50/50 [==============================] - 82s 2s/step - loss: 1059807040.0000 - rmse: 32554.6777 - val_loss: 986088832.0000 - val_rmse: 31402.0508
Epoch 2/120
50/50 [==============================] - 54s 1s/step - loss: 891568896.0000 - rmse: 29859.1504 - val_loss: 984724480.0000 - val_rmse: 31380.3203
Epoch 3/120
50/50 [==============================] - 53s 1s/step - loss: 790001280.0000 - rmse: 28106.9609 - val_loss: 898505856.0000 - val_rmse: 29975.0879
Epoch 4/120
50/50 [==============================] - 53s 1s/step - loss: 762667776.0000 - rmse: 27616.4395 - val_loss: 760849664.0000 - val_rmse: 27583.5039
Epoch 5/120
50/50 [==============================] - 52s 1s/step - loss: 588748032.0000 - rmse: 24264.1309 - val_loss: 587784896.0000 - val_rmse: 24244.2754
Epoch 6/120
50/50 [==============================] - 52s 1s/step - loss: 457812960.0000 - rmse: 21396.5645 - val_loss: 441115104.0000 - val_rmse: 21002.7402
Epoch 7/120
50/50 [==============================] - 51s 1s/step - lo

In [5]:
test_generator.reset()
pred = model.predict(test_generator,steps=len(df_test))

df_submit = pd.DataFrame({'id':df_test['id'],'distance':pred.flatten()})
df_submit.to_csv("submit.csv",index=False)